In [1]:
pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 19.8 MB

In [2]:
import apache_beam as beam

class FilterAndCountNames(beam.PTransform):
    def expand(self, pcoll):
        # Step 1: Filter users based on age
        filtered_users = pcoll | "Filter Users" >> beam.Filter(lambda record: record['age'] >= 25)

        # Step 2: Count the occurrences of each name
        name_counts = (
            filtered_users
            | "Extract Names" >> beam.Map(lambda record: (record['name'], 1))
            | "Count Names" >> beam.CombinePerKey(sum)
        )

        return name_counts


In [7]:
def run_pipeline():
    # Define the pipeline options. For local execution, we use the DirectRunner.
    pipeline_options = beam.options.pipeline_options.PipelineOptions(runner="DirectRunner")

    # Set up the pipeline
    with beam.Pipeline(options=pipeline_options) as p:
        # Read the dataset
        records = (
            p
            | "Read CSV" >> beam.io.ReadFromText('/content/users_dataset.csv', skip_header_lines=1)
            | "Parse CSV" >> beam.Map(lambda line: dict(zip(['user_id', 'name', 'email', 'age'], line.split(','))))
            | "Convert Age to Int" >> beam.Map(lambda record: {**record, 'age': int(record['age'])})
        )

        # Apply the composite transform
        name_counts = records | "Filter and Count Names" >> FilterAndCountNames()

        # Write the results
        name_counts | "Write Results" >> beam.io.WriteToText('/content/name_counts.txt')

    print("Pipeline executed successfully!")

# Uncomment and run the function to execute the pipeline
# run_pipeline()


In [8]:
run_pipeline()

Pipeline executed successfully!
